In [45]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [14]:
train_data = pd.read_csv('LSTM-Multivariate_pollution.csv')
test_data = pd.read_csv('pollution_test_data1.csv')

train_data['wnd_dir'] = train_data['wnd_dir'].map({'SE':0,'cv':1,'NW':2,'NE':3})
test_data['wnd_dir'] = test_data['wnd_dir'].map({'SE':0,'cv':1,'NW':2,'NE':3})

train_data = train_data.drop(columns=['date'])
train_data.head()

,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
0,129.0,-16,-4.0,1020.0,0,1.79,0,0
1,148.0,-15,-4.0,1020.0,0,2.68,0,0
2,159.0,-11,-5.0,1021.0,0,3.57,0,0
3,181.0,-7,-5.0,1022.0,0,5.36,1,0
4,138.0,-7,-5.0,1022.0,0,6.25,2,0


In [26]:
test_data.head()

,dew,temp,press,wnd_dir,wnd_spd,snow,rain,pollution
0,-16,4,1027,0,3.58,0,0,128
1,-17,5,1027,0,7.60,0,0,77
2,-16,4,1027,0,9.39,0,0,65
3,-16,1,1028,1,0.89,0,0,79
4,-14,0,1028,3,1.79,0,0,93


In [34]:
X_train = train_data.iloc[:, 1:]
Y_train = train_data.iloc[:, 1]

X_test = test_data.iloc[:, :-1]
Y_test = test_data.iloc[:, -1]

In [35]:
scaler = StandardScaler()
scaler.fit(X_train)
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [47]:
x_train, x_val, y_train, y_val = train_test_split(scaled_X_train, Y_train, test_size=0.2)

In [43]:
x_train.shape

(35040, 7)

In [46]:
model = Sequential([
    LSTM(7, return_sequences=True),
    Dropout(0.2),
    LSTM(64, return_sequences=True),
    Dropout(0.1),
    LSTM(32, return_sequences=True),
    Dropout(0.2),
    Dense(1, activation='relu')
])
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [49]:
early_stopping = EarlyStopping(monitor='mse', patience=20, verbose=1, restore_best_weights=True)
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=50, verbose=1, batch_size=64, callbacks=[early_stopping])

Epoch 1/50


ValueError: Input 0 of layer "lstm_1" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 7)